In [1]:
import pandas as pd
import numpy as np
import gzip
import json

In [2]:
def parse(path, num_rows):
  g = gzip.open(path, 'rb')
  i = 0
  for l in g:
    yield json.loads(l)
    i += 1
    if i == num_rows:
      break

def getDF(path, num_rows):
  i = 0
  df = {}
  for d in parse(path, num_rows):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

#num_rows = 50000
df = getDF('reviews_Movies_and_TV_5.json.gz', 1500000)

In [3]:
df['reviewText'] = df['reviewText'].str.replace('"','')
df['summary'] = df['summary'].str.replace('"','')

In [4]:
bert_training = df.tail(100000)
df = df.head(1400000)

In [5]:
bert_training.to_pickle('bert_training.pkl')

In [18]:
df = df.rename(columns={'asin':'movieID'})
df

,reviewerID,movieID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
263519,A1127LKNR08JJK,630174411X,acwpython@juno.com,"[0, 4]",The movie was ok but it did lack a few things....,4.0,GREAT,879379200,1997-11-13
17471,A1127LKNR08JJK,0767824571,acwpython@juno.com,"[0, 0]","This movie rules. It has humor, love, suspens...",5.0,Best Movie Ever!!!,879465600,1997-11-14
49404,A1127LKNR08JJK,0783222068,acwpython@juno.com,"[0, 0]",HA HA HA!!! THose were the first words anyone ...,5.0,Hilarious,879465600,1997-11-14
26150,A1127LKNR08JJK,0780022181,acwpython@juno.com,"[1, 2]",This is one of the best movies in the world. ...,5.0,This should get a 15,879897600,1997-11-19
355639,A1127LKNR08JJK,6303094880,acwpython@juno.com,"[2, 3]",This is so funny. It has a lot of good jokes ...,5.0,Awesome,879897600,1997-11-19
...,...,...,...,...,...,...,...,...,...
226605,AX3YHACFZOI40,6300247406,NaN,"[0, 0]",funny and weird,5.0,Five Stars,1405900800,2014-07-21
786104,AX3YHACFZOI40,B0000VCZMK,NaN,"[0, 0]",gross but funny too!,5.0,Five Stars,1405900800,2014-07-21
305722,AX3YHACFZOI40,6302283612,NaN,"[0, 0]",scary great movie,5.0,Five Stars,1405900800,2014-07-21
492742,AX3YHACFZOI40,B000005XN6,NaN,"[0, 0]",took 30 years to find. worth the wait!,5.0,Five Stars,1405900800,2014-07-21


In [15]:
df = df.groupby(['reviewerID']).filter(lambda x: len(x) >= 5)

In [16]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'])

In [17]:
df = df.sort_values(by='reviewTime')
df

,reviewerID,movieID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
263519,A1127LKNR08JJK,630174411X,acwpython@juno.com,"[0, 4]",The movie was ok but it did lack a few things....,4.0,GREAT,879379200,1997-11-13
17471,A1127LKNR08JJK,0767824571,acwpython@juno.com,"[0, 0]","This movie rules. It has humor, love, suspens...",5.0,Best Movie Ever!!!,879465600,1997-11-14
49404,A1127LKNR08JJK,0783222068,acwpython@juno.com,"[0, 0]",HA HA HA!!! THose were the first words anyone ...,5.0,Hilarious,879465600,1997-11-14
26150,A1127LKNR08JJK,0780022181,acwpython@juno.com,"[1, 2]",This is one of the best movies in the world. ...,5.0,This should get a 15,879897600,1997-11-19
355639,A1127LKNR08JJK,6303094880,acwpython@juno.com,"[2, 3]",This is so funny. It has a lot of good jokes ...,5.0,Awesome,879897600,1997-11-19
...,...,...,...,...,...,...,...,...,...
226605,AX3YHACFZOI40,6300247406,NaN,"[0, 0]",funny and weird,5.0,Five Stars,1405900800,2014-07-21
786104,AX3YHACFZOI40,B0000VCZMK,NaN,"[0, 0]",gross but funny too!,5.0,Five Stars,1405900800,2014-07-21
305722,AX3YHACFZOI40,6302283612,NaN,"[0, 0]",scary great movie,5.0,Five Stars,1405900800,2014-07-21
492742,AX3YHACFZOI40,B000005XN6,NaN,"[0, 0]",took 30 years to find. worth the wait!,5.0,Five Stars,1405900800,2014-07-21


In [19]:
df.groupby('reviewerID').cumcount()

263519     0
17471      1
49404      2
26150      3
355639     4
          ..
226605     4
786104     5
305722     6
492742     7
1056025    7
Length: 1303092, dtype: int64

In [20]:
df['reviewer_count'] = df.groupby('reviewerID').cumcount()

In [49]:
df.head(5)

,reviewerID,movieID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviewer_count
263519,A1127LKNR08JJK,630174411X,acwpython@juno.com,"[0, 4]",The movie was ok but it did lack a few things....,4.0,GREAT,879379200,1997-11-13,0
17471,A1127LKNR08JJK,0767824571,acwpython@juno.com,"[0, 0]","This movie rules. It has humor, love, suspens...",5.0,Best Movie Ever!!!,879465600,1997-11-14,1
49404,A1127LKNR08JJK,0783222068,acwpython@juno.com,"[0, 0]",HA HA HA!!! THose were the first words anyone ...,5.0,Hilarious,879465600,1997-11-14,2
26150,A1127LKNR08JJK,0780022181,acwpython@juno.com,"[1, 2]",This is one of the best movies in the world. ...,5.0,This should get a 15,879897600,1997-11-19,3
355639,A1127LKNR08JJK,6303094880,acwpython@juno.com,"[2, 3]",This is so funny. It has a lot of good jokes ...,5.0,Awesome,879897600,1997-11-19,4


In [22]:
train = df.loc[df['reviewer_count']<5]
test = df.loc[df['reviewer_count']>=5]

In [29]:
train = train.reset_index(drop=True)
train

,reviewerID,movieID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviewer_count
0,A1127LKNR08JJK,630174411X,acwpython@juno.com,"[0, 4]",The movie was ok but it did lack a few things....,4.0,GREAT,879379200,1997-11-13,0
1,A1127LKNR08JJK,0767824571,acwpython@juno.com,"[0, 0]","This movie rules. It has humor, love, suspens...",5.0,Best Movie Ever!!!,879465600,1997-11-14,1
2,A1127LKNR08JJK,0783222068,acwpython@juno.com,"[0, 0]",HA HA HA!!! THose were the first words anyone ...,5.0,Hilarious,879465600,1997-11-14,2
3,A1127LKNR08JJK,0780022181,acwpython@juno.com,"[1, 2]",This is one of the best movies in the world. ...,5.0,This should get a 15,879897600,1997-11-19,3
4,A1127LKNR08JJK,6303094880,acwpython@juno.com,"[2, 3]",This is so funny. It has a lot of good jokes ...,5.0,Awesome,879897600,1997-11-19,4
...,...,...,...,...,...,...,...,...,...,...
448565,AX3YHACFZOI40,B000058TIE,NaN,"[0, 0]",funny!,5.0,Five Stars,1405900800,2014-07-21,0
448566,AX3YHACFZOI40,B0000542CM,NaN,"[0, 0]",love him!!!,5.0,Five Stars,1405900800,2014-07-21,1
448567,AX3YHACFZOI40,0792843606,NaN,"[0, 0]",great classic movie,5.0,Five Stars,1405900800,2014-07-21,2
448568,AX3YHACFZOI40,6303588980,NaN,"[0, 0]",fun movie,5.0,Five Stars,1405900800,2014-07-21,3


In [31]:
test = test.reset_index(drop=True)
test

,reviewerID,movieID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviewer_count
0,A1127LKNR08JJK,6302952727,acwpython@juno.com,"[2, 51]",THis is the shortest movie but the funniest. ...,5.0,no,879897600,1997-11-19,5
1,A1127LKNR08JJK,6300248526,acwpython@juno.com,"[0, 4]",This is one of the greatest shows i have ever ...,5.0,why do we have to fill this out?,879897600,1997-11-19,6
2,A1127LKNR08JJK,079283318X,acwpython@juno.com,"[0, 0]",This is the best 007 movie ever. It has a lot...,5.0,Suspensful,882489600,1997-12-19,7
3,A25W58TZK09YET,079074774X,Charles Culbertson (cculbert@cfw.com),"[13, 14]",Errol Flynn made several films about World War...,5.0,Exceptional Combat Drama,892771200,1998-04-17,5
4,A25W58TZK09YET,6302682606,Charles Culbertson (cculbert@cfw.com),"[56, 59]",&quot;The Edge of Darkness&quot; is one of the...,5.0,Gripping World War II Drama,892771200,1998-04-17,6
...,...,...,...,...,...,...,...,...,...,...
854517,A11YIHB6IW352W,B00006FDHI,Theodore J. Jackson Sr.,"[0, 0]",One of my favorite movies. Have watched it sev...,5.0,Five Stars,1405900800,2014-07-21,21
854518,AX3YHACFZOI40,B0000VCZMK,NaN,"[0, 0]",gross but funny too!,5.0,Five Stars,1405900800,2014-07-21,5
854519,AX3YHACFZOI40,6302283612,NaN,"[0, 0]",scary great movie,5.0,Five Stars,1405900800,2014-07-21,6
854520,AX3YHACFZOI40,B000005XN6,NaN,"[0, 0]",took 30 years to find. worth the wait!,5.0,Five Stars,1405900800,2014-07-21,7


In [47]:
train_sample = train.loc[:50000,['reviewerID','movieID','reviewText','summary','overall']]
train_sample

,reviewerID,movieID,reviewText,summary,overall
0,A1127LKNR08JJK,630174411X,The movie was ok but it did lack a few things....,GREAT,4.0
1,A1127LKNR08JJK,0767824571,"This movie rules. It has humor, love, suspens...",Best Movie Ever!!!,5.0
2,A1127LKNR08JJK,0783222068,HA HA HA!!! THose were the first words anyone ...,Hilarious,5.0
3,A1127LKNR08JJK,0780022181,This is one of the best movies in the world. ...,This should get a 15,5.0
4,A1127LKNR08JJK,6303094880,This is so funny. It has a lot of good jokes ...,Awesome,5.0
...,...,...,...,...,...
49996,APSG3FGVOX0DH,6305972052,I just finished whatching a video cassete of t...,fulci's best,4.0
49997,A1MTAPEPFCRMZF,0790741660,The performances in this film are gripping. I...,The Truth Would Have Been Better,3.0
49998,A2JDWQN4JT6H09,6301966554,"Despite this shortcoming, if one can call a la...","What, no special effects?",5.0
49999,A1Y9VYL0X4J3TQ,B000063UYL,We bought this dvd based on the glowing review...,Don't believe the hype!,3.0


In [48]:
train_sample.to_pickle('train_sample.pkl')

In [44]:
test_sample = test.loc[2000,:]
test_sample.to_pickle('test_sample.pkl')

In [9]:
df.groupby(['reviewerID'])['overall'].count().describe()

count    122155.000000
mean         11.460849
std          36.461648
min           1.000000
25%           4.000000
50%           6.000000
75%           9.000000
max        2295.000000
Name: overall, dtype: float64

In [10]:
train = df.loc[df['reviewTime'] <= '2013-01-01']
test = df.loc[df['reviewTime'] > '2013-01-01']

In [11]:
reviewer_count = train.groupby(['reviewerID'])['overall'].count()

In [12]:
reviewer_count = reviewer_count.reset_index().rename(columns={'overall':'reviewer_count'})

In [13]:
movie_count = train.groupby(['movieID'])['overall'].count()

In [14]:
movie_count = movie_count.reset_index().rename(columns={'overall':'movie_count'})
movie_count

,movieID,movie_count
0,0005019281,45
1,0005119367,54
2,0307141985,5
3,0307142469,26
4,0307142477,6
...,...,...
41880,B004EPYZO4,86
41881,B004EPYZOO,108
41882,B004EPYZP8,155
41883,B004EPYZPS,281


In [15]:
test = test.merge(reviewer_count,on='reviewerID',how='left')

In [16]:
test = test.merge(movie_count,on='movieID',how='left')
test['reviewer_count'] = test['reviewer_count'].fillna(0)
test['movie_count'] = test['movie_count'].fillna(0)
test

,reviewerID,movieID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviewer_count,movie_count
0,A3NPBGHQDKBB13,B0001J20QW,Cindy R,"[0, 0]",Great holiday movie. So glad to be able to pur...,5.0,Wonderful!,1357084800,2013-01-02,4.0,30.0
1,A3IXZJ1L73DYTA,B002GSXKQU,Jack E. Levic,"[1, 1]",I've reviewed the four films separately so I j...,4.0,SET OF 4 CLASSIC HOLIDAY FILMS,1357084800,2013-01-02,123.0,38.0
2,A2TUJ8W8UCIROQ,6300216748,chuck,"[0, 1]",Great spy thriller. has a good beginning that...,5.0,Great Movie,1357084800,2013-01-02,1.0,94.0
3,AZFQHNW5FLCMU,B000A3XY8C,Sue E. Devecka,"[0, 0]",I ordered this workout with my amazon.com gift...,5.0,Loved this workout!,1357084800,2013-01-02,2.0,23.0
4,A3UT71DE0NS65Y,0792147618,Otis L. Solomon,"[0, 1]",Great Great mivie in the Alex Cross series pla...,5.0,Kiss the Girls,1357084800,2013-01-02,4.0,66.0
...,...,...,...,...,...,...,...,...,...,...,...
402309,A2P23X9QF6OXUT,B000YQAMZI,"Rachel Dale ""Consider""","[0, 0]",I was in a Meryl Streep mood. I always like se...,4.0,An excellent performance,1405987200,2014-07-22,0.0,27.0
402310,A39UZORCGHK80U,B0000C0FFV,NaN,"[0, 0]",I had been looking for this film for quite awh...,4.0,Thought it was one of Reynolds best fulms.,1405987200,2014-07-22,0.0,8.0
402311,A2HCF3U745WQAK,B000IJ79U2,NaN,"[0, 0]",I got the whole set ( well... 3; I think that...,4.0,I got the whole set ( well... ...,1405987200,2014-07-22,0.0,8.0
402312,A336FCPK8ZVMYT,B000NDDU1K,"Gerald Williams ""booklistener""","[0, 0]",Old people like me need subtitles.,4.0,Four Stars,1405987200,2014-07-22,6.0,3.0
